In [41]:
from torchvision import transforms,datasets
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from torch.functional import F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time
import copy

In [2]:
writer = SummaryWriter('runs/cfiar')

In [3]:
def show_image(img,channel=3):
    if channel==1:
        plot.imshow(img.permute(1,2,0),cmap="gray")
    elif channel==3:
        plot.imshow(img.permute(1,2,0))

In [4]:
def get_stats(path):
    train_dataset = datasets.CIFAR10(path,transform=transforms.Compose([transforms.ToTensor()]))
    trainstack = torch.stack([x for x,y in train_dataset])
    means = [trainstack[:,i,:,:].mean() for i in range(3)]
    stds = [trainstack[:,i,:,:].std() for i in range(3)]
    return means,stds

In [5]:
means,stds = get_stats('data/')

In [6]:
means,stds

([tensor(0.4914), tensor(0.4822), tensor(0.4465)],
 [tensor(0.2470), tensor(0.2435), tensor(0.2616)])

In [8]:
transform_train = transforms.Compose([
                                transforms.RandomResizedCrop(32),
                                transforms.RandomPerspective(),
                                transforms.RandomRotation(90,center=(15,15),fill=means),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(means,stds)])


In [15]:
transfrom_val = transforms.Compose([transforms.ToTensor(),transforms.Normalize(means,stds)])

In [13]:
data = datasets.CIFAR10('data/',train=True,transform=None)
test_dataset = datasets.CIFAR10('data/',train=False,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(means,stds)]))

In [14]:
train_dataset,valid_dataset = torch.utils.data.random_split(data,[40000,10000])

In [20]:
train_dataset.dataset.transform = transform_train
valid_dataset.dataset.transform = transfrom_val

In [22]:
train_dataset.dataset[0][0].shape

torch.Size([3, 32, 32])

In [30]:
train_dls = torch.utils.data.DataLoader(train_dataset,batch_size=bs,shuffle=True)
valid_dls = torch.utils.data.DataLoader(valid_dataset,batch_size=bs,shuffle=True)

In [28]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        in_channel=3
        num_classes=10
        batch_size=256
        ks = 3
        out_channel = 32
        self.conv1 = nn.Conv2d(in_channel,out_channel,ks,padding=1)
        self.batch1 = nn.BatchNorm2d(out_channel)
        self.conv2 = nn.Conv2d(out_channel,out_channel*2,ks,padding=1)
        self.batch2 = nn.BatchNorm2d(out_channel*2)
        self.conv3 = nn.Conv2d(out_channel*2,out_channel*2,ks,padding=1)
        self.batch3 = nn.BatchNorm2d(out_channel*2)
        self.lin1 = nn.Linear(4*4*out_channel*2,128)
        self.batchl1 = nn.BatchNorm1d(128)
        self.lin2 = nn.Linear(128,64)
        self.batchl2 = nn.BatchNorm1d(64)
        self.lin3 = nn.Linear(64,32)
        self.batchl3 = nn.BatchNorm1d(32)
        self.lin4 = nn.Linear(32,num_classes)
        self.batchl4 = nn.BatchNorm1d(10)
        
        
    def forward(self,X):
        conv1 = self.batch1(F.max_pool2d(torch.selu(self.conv1(X)),2))
        conv2 = self.batch2(F.max_pool2d(torch.selu(self.conv2(conv1)),2))
        conv3 = self.batch3(F.max_pool2d(torch.selu(self.conv3(conv2)),2))
#         print(conv3.shape)
        flat = conv3.view(-1,4*4*32*2)
#         print(flat.shape)
        lin1 = self.batchl1(torch.selu(self.lin1(flat)))
        lin2 = self.batchl2(torch.selu(self.lin2(lin1)))
        lin3 = self.batchl3(torch.relu(self.lin3(lin2)))
        out = self.batchl4(torch.relu(self.lin4(lin3)))
#         print(out.shape)
        return out

In [73]:
def learning_loop(data,model,optimizer,loss_fn,epochs):
    
    best_model = copy.deepcopy(model.state_dict())
    for epoch in range(1,epochs+1):
        print(f"Epoch = {epoch}/{epochs}")
        print("-"*15)
        print()
        for phase in data:
            epoch_loss = 0.0
            correct = 0
            total = 0
            accuracy = 0
            for images,labels in data[phase]:
                images,labels = images.to(device),labels.to(device)
                optimizer.zero_grad()
                if phase=="valid":
                    model.eval()
                else:
                    model.train()
                outputs = model(images)
                if phase=="train":
                    loss = loss_fn(outputs,labels)
                    loss.backward()
                    optimizer.step()
                with torch.no_grad():
                    epoch_loss += loss.item()
                    total += len(labels)
                    _,predictions = torch.max(outputs,dim=1)
                    correct += (predictions==labels).sum().item()
            print(f"{phase} Loss == {epoch_loss/len(data[phase]):.4f}")
            print(f"{phase} Accuracy == {correct/total:.2f}")
            if phase=="valid":
                if correct/total > accuracy:
                    best_model = copy.deepcopy(model.state_dict())
                accuracy = correct/total
            else:
                scheduler.step()

        
    return best_model

In [74]:
bs = 128
lr = 0.03
epochs = 15
device = "cuda" if torch.cuda.is_available else "cpu"
classes = data.classes
l_t = len(train_dls)
l_v = len(valid_dls)

In [75]:
datas = {"train":train_dls,"valid":valid_dls}
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,gamma=0.75)
loss_fn = nn.CrossEntropyLoss()

In [76]:
weights = learning_loop(datas,model,optimizer,loss_fn,epochs)

Epoch = 1/15
---------------

train Loss == 1.4594
train Accuracy == 0.47
valid Loss == 1.3226
valid Accuracy == 0.59
Epoch = 2/15
---------------

train Loss == 0.9949
train Accuracy == 0.65
valid Loss == 0.8641
valid Accuracy == 0.67
Epoch = 3/15
---------------

train Loss == 0.8062
train Accuracy == 0.72
valid Loss == 1.2329
valid Accuracy == 0.71
Epoch = 4/15
---------------

train Loss == 0.6656
train Accuracy == 0.77
valid Loss == 0.6558
valid Accuracy == 0.73
Epoch = 5/15
---------------

train Loss == 0.5490
train Accuracy == 0.81
valid Loss == 0.6412
valid Accuracy == 0.74
Epoch = 6/15
---------------

train Loss == 0.4422
train Accuracy == 0.85
valid Loss == 0.5954
valid Accuracy == 0.74
Epoch = 7/15
---------------

train Loss == 0.3487
train Accuracy == 0.88
valid Loss == 0.5333
valid Accuracy == 0.75
Epoch = 8/15
---------------

train Loss == 0.2581
train Accuracy == 0.91
valid Loss == 0.2565
valid Accuracy == 0.75
Epoch = 9/15
---------------

train Loss == 0.1886
train